# Multiword expressions identification and extraction

The task shows two simple methods useful for identifying multiword expressions (MWE) in corpora.

## Tasks

1. Use SpaCy [tokenizer API](https://spacy.io/api/tokenizer) to tokenize the text from the law corpus.

In [81]:
import os
import pickle
import locale
# python -m spacy download en_core_web_sm
# python -m spacy download pl_core_news_sm
import re
import string
import tarfile
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
import morfeusz2
import numpy as np
import pandas as pd
import regex
import spacy
from elasticsearch import *
from elasticsearch.helpers import *
from elasticsearch_dsl import *
from elasticsearch_dsl import query
from spacy.tokenizer import *

matplotlib.style.use("ggplot")
import time
import math
import Levenshtein

%matplotlib inline
import pandas as pd
locale.setlocale(locale.LC_COLLATE, 'pl_PL.UTF-8')


'pl_PL.UTF-8'

2. Compute **bigram** counts of downcased tokens.  Given the sentence: "The quick brown fox jumps over the
   lazy dog.", the bigram counts are as follows:
   
   * "the quick": 1
   * "quick brown": 1
   * "brown fox": 1
   * . ...
   * "dog .": 1

In [53]:
nlp = spacy.load("pl_core_news_sm")
tokenizer = Tokenizer(nlp.vocab)

tokens = {}
tokens_list = []
i = 0
path = "../data/ustawy"
for filename in os.listdir(path):
    with open(os.path.join(path, filename), "r", encoding="utf-8") as file: 
        act = file.read()
        act = regex.sub(r"\s+", " ", act)
        act = act.lower()
        words = [token.text for token in tokenizer(act)]
        tokens[file.name] = words
        tokens_list = tokens_list + words
        i += 1
        if i % 200 == 0:
            print(i)
            


200
400
600
800
1000


In [54]:
tokens_list[0:10]

[' ', 'dz.u.', 'z', '1998', 'r.', 'nr', '117,', 'poz.', '759', 'ustawa']

In [55]:
def separate_puctuations(tokens):
    new_tokens = []
    for token in tokens:
        splitted = regex.findall(r"[\w']+|[.,!?;]", token)  #https://stackoverflow.com/questions/367155/splitting-a-string-into-words-and-punctuation
        new_tokens +=splitted
    return new_tokens

tokens = ['new,','fast,','expensive'] 
separate_puctuations(tokens)

['new', ',', 'fast', ',', 'expensive']

In [56]:
words = [token.text for token in tokenizer(text)]
def bigrams(words):
    words = list(map(lambda x: x.strip(),words))
    words = zip(words, words[1:])
    return [' '.join(pair) for pair in words]

text = "The quick brown fox jumps over the lazy dog."
print(bigrams(words))

['The quick', 'quick brown', 'brown fox', 'fox jumps', 'jumps over', 'over the', 'the lazy', 'lazy dog.']


In [57]:
tokens_list = separate_puctuations(tokens_list)
gram2 = bigrams(tokens_list)

In [60]:
Counter(gram2).most_common(5)


[('art .', 83779),
 ('ust .', 53552),
 ('poz .', 45222),
 ('. 1', 43484),
 (', poz', 43192)]

   
3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries **after**
   computing the bigram counts.
    

In [83]:
# data = gram2.filter()
gram2 =[token for token in gram2 if all(char not in string.punctuation for char in token)]
gram2[0:5]
Counter(gram2).most_common(5)

[('w art', 32045),
 ('mowa w', 28471),
 ('w ust', 23557),
 ('o których', 13884),
 ('których mowa', 13857)]

3306833

<class 'AttributeError'>: 'dict' object has no attribute 'most_common'

4. Use [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) to compute the measure 
   for all pairs of words. 

In [78]:
bigram_freq = Counter(data)
count = sum(bigram_freq.values())
p_bigram = {k: v/count for k, v in bigram_freq.items()}


tokens_freq = Counter(tokens_list)
count = sum(tokens_freq.values())
p_token = {k: v/count for k, v in tokens_freq.items()}
# map(lambda x: x/count,bigram_freq)
# word_freq = Counter(tokens_list

# reduce(list(bigram_freq.values))

In [107]:
import operator


In [109]:
def pmi(x,y): #pointwise_mutual_information
    result = p_bigram[x+" "+y] / (p_token[x] * p_token[y])
    return math.log2(result)
    

bigram_pmis =  {}
for key in gram2:
    if len(key.split())>2:
        print(key)
    bigram_pmis[key] = pmi(*key.split())


    

In [110]:
dict(sorted(bigram_pmis.items(), key=operator.itemgetter(1),reverse=True))

{'korzy stający': 22.804442962406995,
 'gałki ocznej': 22.804442962406995,
 'przedemery talne': 22.804442962406995,
 'organa uchwałodawcze': 22.804442962406995,
 'kropki wstawić': 22.804442962406995,
 'antykonkurencyjnym koncentracjom': 22.804442962406995,
 'skupiających kibiców': 22.804442962406995,
 'chuli gańskich': 22.804442962406995,
 'znająca pjm': 22.804442962406995,
 'przyspo sobieniu': 22.804442962406995,
 'skoczów komorowice': 22.804442962406995,
 'mówił szczerą': 22.804442962406995,
 'uczucie wstydu': 22.804442962406995,
 'nuklidy rozszczepialne': 22.804442962406995,
 'nm nanometrów': 22.804442962406995,
 'aparatami rentgenowskimi': 22.804442962406995,
 'stabilnym jodem': 22.804442962406995,
 'stemplach kontrolerskich': 22.804442962406995,
 'uczynieniu nieczytelnym': 22.804442962406995,
 'odvjetnik odvjetnica': 22.804442962406995,
 'środowiskami przestępczymi': 22.804442962406995,
 'hal aukcyjnych': 22.804442962406995,
 'pompowni bolko': 22.804442962406995,
 'siedlec moszcze

In [95]:
print(*["Ala","ma","aligatora"])

Ala ma aligatora


5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5
   occurrences).

7. Use [log likelihood ratio](http://tdunning.blogspot.com/2008/03/surprise-and-coincidence.html) (LLR) to compute the measure
   for all pairs of words.

8. Sort the word pairs according to that measure in the descending order and display top 10 entries.

9. Compute **trigram** counts for the whole corpus and perform the same filtering.

10. Use PMI (with 5 occurrence threshold) and LLR to compute top 10 results for the trigrams. Devise a method for computing the values, based on the
   results for bigrams.

11. Create a table comparing the methods (separate table for bigrams and trigrams).

12. Answer the following questions:

   a. Why do we have to filter the bigrams, rather than the token sequence?
   
   b. Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?
   
   c. What types of expressions are discovered by the methods.
   
   d. Can you devise a different type of filtering that would yield better results?